In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import tkinter as tk
from tkinter import filedialog, simpledialog

In [2]:
# BROWSE FILE
def browse_file():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename()  # Open the file dialog
    return file_path

# SPEED CALCULATION
def calculate_speed(d):
    global prev_position
    speed = 0
    if prev_position is not None:
        distance = np.linalg.norm(np.array(d) - np.array(prev_position))
        speed = distance * fps
    prev_position = d
    return round(speed,4)

# ANGLE CALCULATION
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return round(angle,4) 

# CHOOSING VIDEO SOURCE
def choose_video_source():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    choice = simpledialog.askstring("Input", "Choose the video source:\n1. Webcam\n2. Video file",
                                    parent=root)
    return choice

def display():
    # Setup status box
    cv2.rectangle(image, (0,0), (200,73), (245,117,16), -1)
        
    # Stage data
    cv2.putText(image, 'Possibility', (15,15), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, hit_chance, 
                (15,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
#     cv2.putText(image, 'Stance', (150,15), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
#     cv2.putText(image, label, 
#                 (150,50), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                             )

In [5]:
# POSE
def ClassifyPose(landmarks, disp=False):
    label = None
    #Get Coordinates
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    
    #Get Angles
    left_elbow_angle = calculate_angle(left_wrist, left_elbow, left_shoulder)
    right_elbow_angle = calculate_angle(right_wrist, right_elbow, right_shoulder)
    left_shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
    right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
    
    # 1. PUNCH
    if left_elbow_angle<45 and right_elbow_angle<45 and left_shoulder_angle<40 and right_shoulder_angle<40:
        label = "Punching"
    
    # 2. Slap
    flag = False
    if left_shoulder_angle>60 and left_shoulder_angle<120:
        flag = True
    if right_shoulder_angle>60 and right_shoulder_angle<120:
        flag = True
    if flag == True:
        label = "Slapping"
    
    if display:
        cv2.rectangle(image, (210,73), (400,73), (245,117,16), -1)
        cv2.putText(image, 'Stance', (220,15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, label, 
                (220,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    else:
        return label

In [7]:
# Ask the user to choose the video source
choice = choose_video_source()

# Initialize video capture based on user's choice
if choice == '1':
    cap = cv2.VideoCapture(0)
    fps = 30
elif choice == '2':
    video_path = browse_file()
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

# Initialise variables
# 1. Curl counter variables
stage = None
# 2. Speed variables
prev_position = None
hit_chance = None


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            #shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            #elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            #Calculate speed
            lspeed = calculate_speed(left_wrist)
            rspeed = calculate_speed(right_wrist)
            
            #Visualise speed
            cv2.putText(image, str(lspeed), 
                           tuple(np.multiply(left_wrist, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(rspeed), 
                           tuple(np.multiply(right_wrist, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
             # Speed limit
            if lspeed > 10:
                hit_chance = "Possible Hit"
            if lspeed<=10:
                hit_chance = "No Hit"
            if rspeed > 10:
                hit_chance = "Possible Hit"
            if rspeed<=10:
                hit_chance = "No Hit"
            
            # Pose
            ClassifyPose(landmarks, disp=True)
            
        except:
            pass
        
        display()       
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()